# 11 Exceptions
**autor: Erik Král ekral@utb.cz**

---

V této prezentaci probereme **Exceptions** a **Dispose** pattern.

## Kontrola chyb programu

- Compile-time errors (static)
    - Statická typová kontrola
- Run-time errors (dynamic)
    - Dynamická typová kontrola
    - Návratové hodnoty funkcí
    - Global Error Indicator
    - Exceptions (výjimky)


Ukázka v jazyce C



```cpp
#define _CRT_SECURE_NO_WARNINGS

#include <stdio.h>
#include <errno.h>

int main()
{
    FILE* pf = fopen("neexistujici.txt", "rb");

    if (pf == NULL)
    {
        fprintf(stderr, "error number: %d\n", errno);
    }
    else
    {
        fclose(pf);
    }
}
```

Ukázka C#

In [ ]:
using System.IO;

string path = "text.txt";

if(File.Exists(path))
{
    try
    {
        string text = File.ReadAllText(path);
    }
    catch (FileNotFoundException ex)
    {
        Console.WriteLine(ex.Message);
        // pokracuji, napisi uzivateli at zkusi jiny soubor
    }
    catch (ArgumentNullException ex)
    {
        Console.WriteLine(ex.Message);
        throw; // znovu vyvolam, radeji tedy ukoncim aplikaci
    }
}


## Exceptions

Pomocí exception komunikujeme v programu především chyby, které vznikají za běhu programu.
Pokud bychom při volání metod chtěli ještě vracet, zda při provedení metody došlo k chybě, tak by zpracování těchto chyb by bylo nepřehledné a nebylo by oddělené od vlastní logiky problému.
Proto je zpracování chyb v programu ve většině objektově orientovaných jazycích oddělené od vlastí logiky problému.

### Původci Exceptions (výjimek) 

Výjimky v .NET mohou být způsobeny:

1. V aplikaci pomocí příkazu throw

    Příkaz throw okamžitě vyvolá výjimku a další příkazy za ním se už neprovedou.

2. Automaticky vygenerované překladačem a vyvolané při neplatném provedení operace

    Tyto výjimky jsou mohou nastat při provedení určitých operací kdy operace nemůže být provedena nebo je neplatná. Příkladem jsou:
    
    - DivideByZeroException, 
    - IndexOutOfRangeException,
    - NullReferenceException,
    - InvalidCastException,
    - StackOverflowException a další.

### Druhy chyb

#### Usage errors

- Pomáhají odhalit chyby programátora které vedou k výjimkám. Typicky například předání neplatné hodnoty argumentu. 
- Těmto chybám jde zabránit opravou kódu a neošetřujeme je pomocí bloku Try-Catch.

#### Execution errors

- Těmto chybám nelze zabránit opravou kódu.
- Příkladem je například čtení poškozeného souboru nebo souboru, který uživatel smazal těsně než jsme jej začali číst.
- **Program errors** - tyto druhy chyb můžeme ošetřit smysluplně programově pomocí Try-Catch, například pokud se nepovede otevřit soubor, tak uživatele vyzveme aby zvolil jinou cestu k souboru.
- **System failures** – tyto druhy chyb nelze ošetřit smysluplně programově, například pokud dojde k chybě JIT překladače.

### Ošetření výjimek

Výjimky bychom měli speciálně ošetřovat jen pokud na ně můžeme nějakým smysluplným speciálním způsobem zareagovat, například vyzvat uživatele k zadání jiné cesty k souboru. Ostatní výjimky můžeme ošetřit na nejvyšší úrovni aplikace.
Ve vláknech bychom měli ale ošetřovat všechny výjimky.


Příklad vyvolání výjimky

In [ ]:
class BankovniUcet
{
    private decimal zustatek;

    public void Vloz(decimal castka)
    {
        if(castka <= 0)
        {
            throw new ArgumentOutOfRangeException("Zustatek musi byt kladny");
        }

        zustatek += castka;
    }

    public void Vyber(decimal castka)
    {
        if (castka <= 0 ||  castka > zustatek)
        {
            throw new ArgumentOutOfRangeException("Zustatek musi byt kladny a mensi nez zustatek");
        }

        zustatek -= castka;
    }
}


Příklad zachycení výjimky

In [ ]:
using System.Net.Http;

string url = "https://geek-jokes.sameerkumar.website/api?format=json";

HttpClient client = new HttpClient();

try
{
    string jsonString = await client.GetStringAsync(url);
    Console.WriteLine(jsonString);
}
catch(HttpRequestException ex)
{
    Console.WriteLine(ex.Message);
}
finally
{
    client.Dispose();
}


# Dispose Pattern

Slouží k bezpečnému uvolnění zdrojů (paměti, připojení k serveru, k databázi atd.) i v případě výjimky.
V jazyce C++ ke stejnému účelu používáme destruktor, který se zavolá automaticky předtím, než se uvolní objekt z paměti.
V jazyce C# sice máme destruktor také, ale nevíme kdy přesně se zavolá, protože nevíme kdy Garbage Collector uvolní paměť objektu.

Použiti Dispose patternu

In [ ]:
using System.Net.Http;

string url = "https://geek-jokes.sameerkumar.website/api?format=json";

using (HttpClient client = new HttpClient())
{ 
    try
    {
        string jsonString = await client.GetStringAsync(url);
        Console.WriteLine(jsonString);
    }
    catch (HttpRequestException ex)
    {
        Console.WriteLine(ex.Message);
    }
}
